In [1]:
import xarray as xr
import dask
import numpy as np
from datetime import datetime, timedelta
from cartopy.feature.nightshade import Nightshade
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import os, glob, sys, gc
from tqdm.auto import tqdm
import pandas as pd

/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import argparse
import glob
import os
from datetime import timedelta

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import xarray as xr

import importlib

import warnings

In [3]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

In [4]:
from dask.diagnostics import ProgressBar

In [5]:
import utility_programs.filters as filters
from utility_programs.plotting_routines import panel_plot
from utility_programs.read_routines.GITM import auto_read as auto_read_gitm
from utility_programs.read_routines.GITM import gitm_times_from_filelist
from utility_programs.read_routines.SAMI import auto_read as auto_read_sami
from utility_programs import plotting_routines
from utility_programs.utils import ut_to_lt
from utility_programs.utils import add_lt_to_dataset

/glade/u/home/abukowski/Documents/CEDAR2023/utility_programs/plotting_routines.py:13: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = geopandas.read_file(geopandas.datasets.get_path("naturalearth_lowres"))


In [6]:
# cluster = PBSCluster(queue='regular',
#                      account='UTAD0001',
#                      local_directory='$TMPDIR',
#                      cores=5,
#                      processes=5,
#                      memory='109GB',
#                      log_directory='/glade/u/home/abukowski/dask-worker-space/',
#                      scheduler_options={'dashboard_address': ':44511'},
#                      walltime='01:00:00',
#                      resource_spec='select=1:ncpus=24:mem=109GB')

In [7]:
cluster=PBSCluster()

In [8]:
cluster.adapt(maximum_jobs=2)
client = Client(cluster)


In [9]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.7:44669,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [10]:
data = '/glade/u/home/abukowski/scratch/postprocessed/'

In [11]:
gitms = {}
samis = {}

for full_path in np.sort(glob.glob(os.path.join(data,'*.nc'))):

    fname = full_path[full_path.rfind('/')+1:]
    datas = fname[:-3].split('_')
    month = datas[0]

    if 'bz' in datas[1]:
        if 'SAMI' in datas:
            model = datas[-2] + '-'+datas[-1]
        else:
            model = datas[-1]
        month = datas[0]
        runname = datas[1][5:]
        onset = datas[2]
        duration = datas[3]

    elif 'thm' not in datas:
        if 'SAMI' in datas:
            model = datas[-2] + '-'+datas[-1]
        else:
            model = datas[-1]
        month = datas[0]
        runname = datas[1]
        onset = datas[2]
        duration = datas[3]
        
    else:
        continue

    if model == 'GITM':
        gitms['-'.join([month, onset, duration, runname])] = xr.open_mfdataset(full_path, engine='h5netcdf')

    elif model == 'SAMI-REGRID':
        samis['-'.join([month, onset, duration, runname])] = xr.open_mfdataset(full_path, engine='h5netcdf')


In [12]:
gitms.keys()

dict_keys(['may-ut12-storm-1hr-full', 'may-ut12-storm-on-full', 'may-ut18-storm-on-full', 'may-ut6-storm-1hr-full', 'may-ut6-storm-on-full', 'may-ut12-storm-1hr-half', 'may-ut12-storm-on-half', 'may-ut12-storm-on-quarter', 'nov-ut12-storm-on-full', 'nov-ut18-storm-on-full', 'nov-ut6-storm-on-full', 'nov-ut12-storm-on-half', 'nov-ut12-storm-on-bz15-byneg', 'nov-ut12-storm-on-bz15-bypos', 'nov-ut12-storm-on-bz17', 'nov-ut12-storm-on-bz8-byneg', 'nov-ut12-storm-on-bz8-bypos'])

In [13]:
samis.keys()

dict_keys(['may-ut12-storm-on-full', 'may-ut18-storm-on-full', 'may-ut6-storm-on-full', 'may-ut12-storm-on-half', 'may-ut12-storm-on-quarter', 'nov-ut12-storm-on-full', 'nov-ut18-storm-on-full', 'nov-ut6-storm-on-full', 'nov-ut12-storm-on-half', 'nov-ut12-storm-on-bz15-byneg', 'nov-ut12-storm-on-bz15-bypos', 'nov-ut12-storm-on-bz17', 'nov-ut12-storm-on-bz8-byneg', 'nov-ut12-storm-on-bz8-bypos'])

In [14]:
# Preprocess


## Time slicing
t_starts = {'ut6':'03:00:00', 'ut12':'09:00:00', 'ut18':'15:00:00'}

for k in gitms.keys():
    key = k.split('-')[1]
    t_start = gitms[k].time.values[0] + pd.Timedelta('1 day' + t_starts[key])
    t_end = t_start + pd.Timedelta('8 hours')
    
    gitms[k] = gitms[k].sel(time = slice(t_start, t_end))
    
for k in samis.keys():
    key = k.split('-')[1]
    t_start = samis[k].time.values[0] + pd.Timedelta('1 day' + t_starts[key])
    t_end = t_start + pd.Timedelta('8 hours')
    samis[k] = samis[k].sel(time = slice(t_start, t_end))    

## TEC calculation in SAMI:
for modelname, ds in samis.items():
    # if 'REGRID' in modelname:
    ds['tec'] = ds.edens.integrate('alt')
    

In [15]:

def custom_panels_keos(da,
                       numplots=8,
                       sel_col='localtime',
                       max_per_row=4,
                       suptitle=None,
                       vmin=None,
                       vmax=None,
                       sharex=True,
                       sharey=True,
                       x='time',
                       cmap='rainbow',
                       single_colorbar=True,
                       colorbar_label=''):
    
    sys.path.append('/glade/u/home/abukowski/SAMI3-GITM-python/')

    if sel_col == 'localtime' and sel_col not in da.coords:
        # print('adding')
        da = add_lt_to_dataset(da, localtimes=90)
    # print(da.coords)
    # return da

    nrows = int(np.ceil(numplots/max_per_row))
    ncols = max_per_row

    f, axs = plt.subplots(nrows,
                          ncols,
                          figsize=(5*nrows, 1.3*ncols if suptitle is not None else 1*ncols),
                          sharey=sharey,
                          sharex=sharex)

    sel_list = np.linspace(da[sel_col].min().values,
                           da[sel_col].max().values,
                           numplots+2)[1:-1]
    
    if single_colorbar:
        if vmin is None:
            vmin = da.min().compute()
        if vmax is None:
            vmax = da.max().compute()
            
    for a, ax in enumerate(axs.flatten()):
        ims = da.sel({sel_col: sel_list[a]}, method='nearest').plot(
            x=x, ax=ax, cmap=cmap,vmin=vmin, vmax=vmax, add_colorbar=not single_colorbar)
        
    if single_colorbar:
        divider = make_axes_locatable(axs[nrows-1, ncols-1])
        cax = divider.append_axes('right', size='5%', pad=0.05, in_layout=True)
        f.colorbar(ims, cax=cax, orientation='vertical', label=colorbar_label)

        divider = make_axes_locatable(axs[0, ncols-1])
        cax = divider.append_axes('right', size='5%', pad=0.05, in_layout=True)
        f.colorbar(ims, cax=cax, orientation='vertical', label=colorbar_label)

    if suptitle is not None:
        f.suptitle(suptitle)
    f.tight_layout()

    return f

In [16]:
def loop_all_keos(ds, runname, model):
    
    sdir='/glade/u/home/abukowski/plots/cedar/exploring/'
    alts = [200, 350, 500, 750, 1000]
    
    for var in ds.data_vars:
        if 'Alt' not in var and 'alt' in ds[var].coords:
            suptitle = 'Mean %s from %s run %s' %(var, model, runname)
            fig = custom_panels_keos(da=ds[var].mean('alt'),
                        suptitle=suptitle)
            try:
                fig.savefig(os.path.join(sdir, 'all-%s-keos' %model, runname, 'alt-mean', var)#.compute()
            except FileNotFoundError:
                os.makedirs(os.path.join(sdir, 'all-%s-keos' %model, runname, 'alt-mean', var))#.compute()
                fig.savefig(os.path.join(sdir, 'all-%s-keos' %model, runname, 'alt-mean', var))#.compute()

            for a in alts:
                suptitle = '%s from %s run %s at Alitude=%i km' %(var, model, runname, int(a))
                fig = custom_panels_keos(da=ds[var].sel(alt=a, method='nearest'),
                        suptitle=suptitle)
                try:
                    fig.savefig(os.path.join(sdir, 'all-%s-keos' %model, runname, 'alt-mean', var))#.compute()
                except FileNotFoundError:
                    os.makedirs(os.path.join(sdir, 'all-%s-keos' %model, runname, 'alt-mean', var))#.compute()
                    fig.savefig(os.path.join(sdir, 'all-%s-keos' %model, runname, 'alt-mean', var))#.compute()
            plt.close()

    return

In [44]:
def makeJH_Plots(ds, runname):
    fig = plotting_routines.loop_panels(ds['AltIntJouleHeating_W_m2'],
                                  ncols=2, title='Joule Heating (W/m2) from ' + runname,
                                  start_time= ds.time.values[0] + pd.Timedelta('2 hours'), time_delta='4 hours',
                                  col_names = ['Before', 'After'], cmap='rainbow', mask_dials = .001,
                                  norm=colors.LogNorm(vmin = .0009,
                                                      vmax = ds['AltIntJouleHeating_W_m2'].max().values))
    
    # os.makedirs()
    fig.savefig('/glade/u/home/abukowski/plots/cedar/exploring/JHbefore-after/%s'%runname)
    plt.close()
    return

In [40]:
def maps_and_dials_just_gitm(ds, runname):
    alts = [150, 250, 350, 450, 650]
    
    for var in ds.data_vars:
        if 'alt' in ds[var].coords:
            for a in alts:

                fig = plotting_routines.map_and_dials(dial_da=ds.AltIntJouleHeating_W_m2,
                                                    total=6,
                                                    map_da=filters.filter_xarray_DA_diff(
                                                        ds[var].sel(alt=a, method='nearest'), percent=False, label='lower', order=2),
                                                    max_per_row=3, mask_dials=0.002,
                                                    suptitle='Joule Heating and %s at %i km' %(var, a),
                                                    time_start=ds.time.values[0] + pd.Timedelta('2.5 hours'),
                                                    map_cmap='bwr', vmin_dial=0.03, vmax_dial=.11)
                try:
                    fig.savefig('/glade/u/home/abukowski/plots/cedar/exploring/JHbefore-after/%s/%s'%(runname, var))#.compute()
                except FileNotFoundError:
                    os.makedirs('/glade/u/home/abukowski/plots/cedar/exploring/JHbefore-after/%s/%s'%(runname))
                    fig.savefig('/glade/u/home/abukowski/plots/cedar/exploring/JHbefore-after/%s/%s'%(runname, var))#.compute()
                plt.close()
    return

In [41]:
def maps_and_dials_any_var(ds, JH_da, runname, model):
    alts = [150, 250, 350, 450, 650]
    
    for var in ds.data_vars:
        if 'alt' in ds[var].coords:
            for a in alts:
                fig = plotting_routines.map_and_dials(dial_da=JH_da.AltIntJouleHeating_W_m2,
                                                    total=6,
                                                    map_da=filters.filter_xarray_DA_diff(
                                                        ds[var].sel(alt=a, method='nearest'), percent=False, label='lower', order=2),
                                                    max_per_row=3, mask_dials=0.002,
                                                    suptitle='Joule Heating and %s at %i km' %(var, a),
                                                    time_start=ds.time.values[0] + pd.Timedelta('2.5 hours'),
                                                    map_cmap='bwr', vmin_dial=0.03, vmax_dial=.11)
                try:
                    fig.savefig('/glade/u/home/abukowski/plots/cedar/exploring/MAP-%s/%s/%s'%(model, runname, var))#.compute()
                except FileNotFoundError:
                    os.makedirs('/glade/u/home/abukowski/plots/cedar/exploring/MAP-%s/%s/'%(model, runname))
                    fig.savefig('/glade/u/home/abukowski/plots/cedar/exploring/MAP-%s/%s/%s'%(model, runname, var))#.compute()
                plt.close()
    return

In [20]:
gitm_temperature_keos = client.map(loop_all_keos, gitms.values(), list(gitms.keys()), ['GITM' for i in gitms.keys()])

In [21]:
sami_temperature_keos = client.map(loop_all_keos, samis.values(), list(samis.keys()), ['SAMI' for i in samis.keys()])

In [45]:
jhplots = client.map(makeJH_Plots, gitms.values(), gitms.keys())

In [37]:
gallmaps = client.map(maps_and_dials_any_var, gitms.values(), gitms.values(), gitms.keys(), ['GITM' for i in gitms.keys()])

In [38]:
newsamis = []

for i in samis.keys():
    if i in gitms.keys():
        newsamis.append(client.submit(maps_and_dials_any_var, samis[i], gitms[i], i, 'SAMI'))

In [49]:
gitms['may-ut12-storm-on-full']

2023-06-20 19:01:04,797 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x7f7db5e6f0d0>>
Traceback (most recent call last):
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/tornado/ioloop.py", line 919, in _run
    val = self.callback()
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/bokeh/server/tornado.py", line 781, in _keep_alive
    c.send_ping()
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/bokeh/server/connection.py", line 93, in send_ping
    self._socket.ping(str(self._ping_count).encode("utf-8"))
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/tornado/websocket.py", line 439, in ping
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
2023-06-20 19:01:05,516 - tornado.application - ERROR - Exception in callback <bound method BokehTorn

<xarray.Dataset>
Dimensions:                          (time: 96, lon: 90, lat: 180, alt: 50)
Coordinates:
  * time                             (time) datetime64[ns] 2011-05-21T09:00:0...
  * lon                              (lon) float64 2.0 6.0 10.0 ... 354.0 358.0
  * lat                              (lat) float64 -89.5 -88.5 ... 88.5 89.5
  * alt                              (alt) float64 100.0 101.7 ... 729.0 751.5
Data variables: (12/49)
    LocalTime                        (time, lon, lat) float64 dask.array<chunksize=(96, 90, 180), meta=np.ndarray>
    SolarZenithAngle                 (time, lon, lat) float64 dask.array<chunksize=(96, 90, 180), meta=np.ndarray>
    VerticalTEC                      (time, lon, lat) float64 dask.array<chunksize=(96, 90, 180), meta=np.ndarray>
    AltIntJouleHeating_W_m2          (time, lon, lat) float64 dask.array<chunksize=(96, 90, 180), meta=np.ndarray>
    AltIntHeatingTransfer_W_m2       (time, lon, lat) float64 dask.array<chunksize=(96, 90, 180), meta=np.ndarray>
    AltIntEuvHeating_W_m2            (time, lon, lat) float64 dask.array<chunksize=(96, 90, 180), meta=np.ndarray>
    ...                               ...
    e_                               (time, lon, lat, alt) float64 dask.array<chunksize=(96, 90, 180, 50), meta=np.ndarray>
    eTemperature                     (time, lon, lat, alt) float64 dask.array<chunksize=(96, 90, 180, 50), meta=np.ndarray>
    iTemperature                     (time, lon, lat, alt) float64 dask.array<chunksize=(96, 90, 180, 50), meta=np.ndarray>
    Vi_east                          (time, lon, lat, alt) float64 dask.array<chunksize=(96, 90, 180, 50), meta=np.ndarray>
    Vi_north                         (time, lon, lat, alt) float64 dask.array<chunksize=(96, 90, 180, 50), meta=np.ndarray>
    Vi_up                            (time, lon, lat, alt) float64 dask.array<chunksize=(96, 90, 180, 50), meta=np.ndarray>
Attributes:
    version:              21.14
    dropped-ghost-cells:  False
    with_time:            True

In [33]:
jhplots[0].result()

AttributeError: 'Dataset' object has no attribute 'AltIntJouleHeating_W_m2'

In [35]:
newsamis[-1].result()

AttributeError: 'NoneType' object has no attribute 'compute'

In [24]:
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -q regular
#PBS -A UTAD0001
#PBS -l select=1:ncpus=36:mem=109GB
#PBS -l walltime=02:00:00

/glade/work/abukowski/conda-envs/SAMI3-GITM/bin/python -m distributed.cli.dask_worker tcp://10.148.0.7:44669 --nthreads 6 --nworkers 6 --memory-limit 16.92GiB --name dummy-name --nanny --death-timeout 60 --interface ib0



In [60]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.7:44669,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,Total threads: 0
Started: 2 hours ago,Total memory: 0 B


In [61]:
gitm_temperature_keos, sami_temperature_keos, jhplots, gallmaps, newsamis

([<Future: error, key: loop_all_keos-f170db34c57e85e833a125c941000360>,
  <Future: pending, key: loop_all_keos-9182bcfdce8ed5c16a58de54d6437d0b>,
  <Future: pending, key: loop_all_keos-94d4030e688a754b636181cec7146ae2>,
  <Future: pending, key: loop_all_keos-7af36ed3813d80942ce4d65d497f029c>,
  <Future: pending, key: loop_all_keos-5eec864a553ece9de43807380d7b820d>,
  <Future: pending, key: loop_all_keos-6db2a71f39c69a7f62e3e0e30bf8837d>,
  <Future: pending, key: loop_all_keos-1c1123a2105564174e8f68e0e3518e3e>,
  <Future: error, key: loop_all_keos-26ea2a4eae1e5d1fd8acc66878c8bbef>,
  <Future: error, key: loop_all_keos-741bf417d6557fc1e2687d2a26f68108>,
  <Future: error, key: loop_all_keos-54f04f89aeaa998f3f9e4ab154371416>,
  <Future: error, key: loop_all_keos-eff94b37c1141ba9b4f6add7dffb24bd>,
  <Future: error, key: loop_all_keos-e770521aa172a01646685b6e1917ea5d>,
  <Future: error, key: loop_all_keos-d7e91bd24c573978467964142ba53c16>,
  <Future: error, key: loop_all_keos-e4d06139766a258

2023-06-21 08:40:55,508 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x7f7db5e6f0d0>>
Traceback (most recent call last):
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/tornado/ioloop.py", line 919, in _run
    val = self.callback()
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/bokeh/server/tornado.py", line 781, in _keep_alive
    c.send_ping()
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/bokeh/server/connection.py", line 93, in send_ping
    self._socket.ping(str(self._ping_count).encode("utf-8"))
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/tornado/websocket.py", line 439, in ping
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError


In [62]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.7:44669,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,Total threads: 0
Started: 14 hours ago,Total memory: 0 B


In [48]:
gitm_temperature_keos[7].result()

2023-06-20 18:56:14,216 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x7f7db5e6f0d0>>
Traceback (most recent call last):
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/tornado/ioloop.py", line 919, in _run
    val = self.callback()
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/bokeh/server/tornado.py", line 781, in _keep_alive
    c.send_ping()
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/bokeh/server/connection.py", line 93, in send_ping
    self._socket.ping(str(self._ping_count).encode("utf-8"))
  File "/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/tornado/websocket.py", line 439, in ping
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
2023-06-20 18:56:14,906 - tornado.application - ERROR - Exception in callback <bound method BokehTorn

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
newsamis

In [ ]:
gallmaps[6].result()

In [ ]:
len(gitms.keys()), len(gitms.values()), len(samis.keys()), len(samis.values())

In [ ]:
gitms.keys()

In [ ]:
gitm_temperature_keos[0]

In [ ]:
gitm_temperature_keos

In [ ]:
client

In [ ]:
import s3g